In [14]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import random

# Preparing the Data

The sensors were sampled at 50Hz, and we want to classify at 1 second resolution, so each training example has 50 timesteps. One problem was that the 'sitting' and 'standing' activity timestamps were corrupted and all displayed the same time. However, going over the rest of the data, it was clear that most 50-sample slices were already chronologically ordered, so assuming that the sampe slices for the 'sitting' and 'standing' activities were already chronologically ordered as well is not a far stretch. Concretely, only 32 instances arise where, in a 50-sample slice, timestamp(A)>timestamp(B) when index(A)<index(B). Therefore, we assume that the samples for the 'sitting' and 'standing' activities are already chronologically ordered.

We don't preprocess the features in any way, but we do add additional features - namely the magnitude of the vectors generated by each of the sensors (root of the sum of squares of the x,y, and z values). This idea was adopted from the original paper that accompanied this dataset.

To create the train, val, and test sets, we split the data by activity label and perform stratified sampling to ensure a good train/val/test split.

In [15]:
print 'Importing data'
data_dir = 'Activity_Recognition_DataSet/'
sources = ['Arm','Belt','Pocket','Wrist']
keys = { 'Standing': 0, 'Sitting': 1, 'Downstairs': 2, 'Upstairs': 3, 'Running': 4, 'Walking': 5}
data = [ [] for i in range( len( keys ) )]
num_classes = len(keys)
time_steps = 50
skipped_samples = 0
for idx in range( len( sources ) ):
  record = pd.read_excel( data_dir + sources[idx] + '.xlsx')
  record.sort_values( 'Time_Stamp', inplace= True)
  for idx2 in range( len( sources ) ):
    record[ sources[idx2] ] = 1 if idx2 == idx else 0

  record['Activity_Label'] = record['Activity_Label'].apply( lambda x: keys[x])
  labels = record['Activity_Label']
  record.drop(labels=['Activity_Label'], axis=1,inplace = True)
  record['A'] = ( record['Ax']**2 + record['Ay']**2 + record['Az']**2 ) **0.5
  record['G'] = ( record['Gx']**2 + record['Gy']**2 + record['Gz']**2 ) **0.5
  record['M'] = ( record['Mx']**2 + record['My']**2 + record['Mz']**2 ) **0.5

  for sample_idx in range(0, len(labels), time_steps):
    if sample_idx + time_steps - 1 < len(labels):
      time_diff = record['Time_Stamp'][sample_idx+time_steps - 1] - record['Time_Stamp'][sample_idx]
      if time_diff <= time_steps/50.0 * 1000 \
          and time_diff >= 0 \
          and labels[sample_idx: sample_idx + time_steps].min() == labels[sample_idx: sample_idx + time_steps].max():
        sample_x = record.iloc[sample_idx: sample_idx + time_steps]
        data[ labels[sample_idx] ].append( ( sample_x.as_matrix(), labels[sample_idx] ) )
      else:
        skipped_samples = skipped_samples + 1
    else:
      skipped_samples = skipped_samples + 1

print 'skipped ' + str(skipped_samples) + ' samples'

train_set, val_set, test_set = [],[],[]

for idx in range( len(data) ):
  label_data = data[idx]
  random.shuffle(label_data)
  train_set.extend( label_data[0: int(0.7*len(label_data))] )
  val_set.extend( label_data[int(0.7*len(label_data)): int(0.8*len(label_data))] )
  test_set.extend( label_data[int(0.8*len(label_data)):] )

  print keys.keys()[idx] + ' examples: train %d val %d test %d'%(int(0.7*len(label_data)),
                                                                  int(0.1*len(label_data)),
                                                                  int(0.2*len(label_data)))
random.shuffle(train_set)
random.shuffle(val_set)
random.shuffle(test_set)
train_X, train_Y = zip( *train_set )
val_X, val_Y = zip( *val_set )
test_X, test_Y = zip( *test_set )

train_X, train_Y = np.stack(train_X), np.array(train_Y)
val_X, val_Y = np.stack(val_X), np.array(val_Y)
test_X, test_Y = np.stack(test_X), np.array(test_Y)

print '%d training samples'%(train_X.shape[0])
print '%d val samples'%(val_X.shape[0])
print '%d test samples'%(test_X.shape[0])


Importing data
skipped 1302 samples
Standing examples: train 1521 val 217 test 434
Walking examples: train 1533 val 219 test 438
Upstairs examples: train 1025 val 146 test 293
Sitting examples: train 949 val 135 test 271
Running examples: train 1549 val 221 test 442
Downstairs examples: train 1532 val 218 test 437
8109 training samples
1161 val samples
2319 test samples


# Building the Model
In most of the literature in this area, we see that neural network techniques come out on top relative to other classifiers. Therefore we will attempt to use more recent techniques - specifically, we will use a LSTM layer followed by two fully-connected layers. Additionally, we add batch normalization layers at the input and between the two fully-connected layers to normalize the input and hidden state. With this setup, the model achieves a test accuracy of about 87%.